In [ ]:
import numpy as np
from scipy import signal
from scipy import fft
import matplotlib.pyplot as plt
%matplotlib widget
#%matplotlib qt

In [ ]:
fs = 3.7e9
numtaps = 127  # Filter order
trans_width = 50e6    # Width of transition from pass to stop, Hz
band = [1e9, 1.8e9]  # Desired pass band, Hz
edges = [0, band[0] - trans_width, band[0],
            band[1], band[1] + trans_width, 0.5*fs]
taps = signal.remez(numtaps, edges, [0, 1, 0], fs=fs)

(w,Hr) = signal.freqz(taps, worN=5000, fs=fs)

In [ ]:
fig = plt.figure(figsize=(20, 5))
ax0 = fig.add_subplot(211)
ax0.stem(np.arange(len(taps)), taps)
ax0.grid(True)
ax0.set_title('Impulse Response')
ax1 = fig.add_subplot(212)
ax1.plot(w, 20*np.log10(abs(Hr)))
ax1.set_xlim((0,fs/2))
ax1.set_xticks((0, fs/4, *band, fs/2))
ax1.set_title('Freq Response')
ax1.grid(True)
fig.savefig('BF_feed_model.png')

In [ ]:
N = 2**20
nfft = 2048
noise_power = 1
n = np.arange(N)
time = n / float(fs)
mod = 2e4*np.cos(2*np.pi*10000*time)
amp = 0.5
carrier = amp * np.sin(2*np.pi*1.25e9*time + mod)
noise = np.random.normal(scale=np.sqrt(noise_power), size=time.shape)
noise *= np.exp(-time/5)
x = carrier + noise

LF_feed_noise = signal.filtfilt(taps, 1, x)

f, t, Sxx = signal.spectrogram(LF_feed_noise, nfft = 2048, fs=fs, return_onesided=False)
median_power = np.median(Sxx[800:900,:].flatten())

In [ ]:
f, t, Sxx = signal.spectrogram(LF_feed_noise, nfft = nfft, fs=fs, return_onesided=False)

def spec(f, t, Sxx):
  fig = plt.figure(figsize=(10, 10))
  ax0 = fig.add_subplot(111)
  #ax0.pcolormesh(t, f, Sxx, shading='gouraud')
  ax0.imshow(fft.fftshift(Sxx, axes=0),
             interpolation='nearest',
             aspect='auto',
             origin='lower',
             extent=(t[0], t[-1], -fs/2, fs/2),
             vmin=0,
             vmax=5*median_power)
  ax0.set_ylabel('Frequency [Hz]')
  ax0.set_xlabel('Time [sec]')
  plt.show()

In [ ]:
spec(f, t, Sxx)

In [ ]:
fc = 1.1e9
L0_len = 1024
par_samples = 16
kc = np.round(fc/fs * L0_len)
#kc = 100
print(kc)
L0 = np.exp(2j*np.pi*(-kc) * n / L0_len)
mixer_out = LF_feed_noise * L0

f, t, Sxx = signal.spectrogram(mixer_out, nfft = 2048, fs=fs)
spec(t, f, Sxx)

In [ ]:
L0_mem = L0[:L0_len*par_samples]
fig = plt.figure(figsize=(20, 5))
ax0 = fig.add_subplot(111)
ax0.stem(L0_mem.real, 'b')
ax0.stem(L0_mem.imag, 'r')

In [ ]:
L0_mem = L0_mem.reshape((L0_len, par_samples))
fig = plt.figure(figsize=(20, 5))
ax0 = fig.add_subplot(211)
ax1 = fig.add_subplot(212)
for mem in range(par_samples):
  ax0.plot(L0_mem[:, mem].real + mem, '.-')
  P_L0=mem = 1/nfft * fft.fft(L0_mem[:, mem])
  ax1.plot(np.abs(P_L0)+mem, '.')

In [ ]:
NCO_w = 16
NCO_dp = 15
Max_NCO_val = 2**(NCO_dp)-1
ReIm = 2
L0_mem_ReIm = L0_mem.view(np.float64).reshape((L0_len, par_samples, ReIm))
L0_mem_ImRe = L0_mem_ReIm[:,:,-1::-1]
L0_mem_ImRe_int = np.round(L0_mem_ImRe * Max_NCO_val)
L0_mem_ImRe_int16 = L0_mem_ImRe_int.astype(np.int16)
L0_mem_uint32 = L0_mem_ImRe_int16.view(np.uint32)[...,0]